In [ ]:
#In here I think the model overfitted. So try to train the model again with 5 Epoches.

In [52]:
from PIL import Image
import numpy as np
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt

In [53]:
# Load the image you want to test
image_path = 'airplne.jpg'  # Replace with the path to your image
image = Image.open(image_path)

In [54]:
# Preprocess the image: resize, convert to tensor, normalize, etc.
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 (EfficientNet input size)
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

image = transform(image).unsqueeze(0)  # Add batch dimension (1, C, H, W)

In [55]:
# Check the image tensor shape
print(f"Image tensor shape: {image.shape}")

Image tensor shape: torch.Size([1, 3, 224, 224])


In [56]:
print(torch.cuda.is_available())

True


In [57]:
# Check if CUDA is available and set the device accordingly
device = torch.device("cpu")
image = image.to(device)  # Move image to the correct device
print(f"Using device: {device}")

Using device: cpu


In [58]:
# Load the pre-trained EfficientNet model (e.g., EfficientNetB0)
model = models.efficientnet_b0(pretrained=False)  # Set pretrained=False since we're using our custom-trained weights


c:\Users\Tuf\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Tuf\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [59]:
# Modify the model's final layer to match CIFAR-10 (10 classes)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 10)

In [60]:
# Load the trained model weights
model.load_state_dict(torch.load('efficientnet_cifar10.pth'))
model = model.to(device)  # Move the model to the same device (GPU or CPU)

C:\Users\Tuf\AppData\Local\Temp\ipykernel_22492\454683468.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('efficientnet_cifar10.pth'))


In [61]:
# Set the model to evaluation mode with no gradient computation
with torch.no_grad():
    model.eval()  # Set to evaluation mode

In [62]:
# Perform inference
with torch.no_grad():
    output = model(image)

In [63]:
# Get the predicted class (highest probability)
_, predicted_class = torch.max(output, 1)

In [64]:
# Map the predicted class to CIFAR-10 class label
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
predicted_label = class_names[predicted_class.item()]

In [65]:
# Output the predicted label
print(f"Predicted class: {predicted_label}")

Predicted class: airplane
